In [2]:
import numpy as np
import pandas as pd
import json
import re

import shinra_util

In [25]:
with open("../output/use.json") as f:
    predicted_use = json.load(f)

with open("../output/type.json") as f:
    predicted_type = json.load(f)

with open("../output/production.json") as f:
    predicted_production = json.load(f)

with open("../output/material.json") as f:
    predicted_material = json.load(f)

with open("../output/characteristic.json") as f:
    predicted_characteristic = json.load(f)

with open("../data/compound_train.json") as f:
    train_raw = json.load(f)['entry']

In [4]:
def contains_set(train, predicted):
    contained = \
    [[t, p] for p in sub_set(train, predicted) for t in train \
            if re.search(clean_txt(p), t) or re.search(clean_txt(t), p)]
    contained = np.array(contained)
    if len(contained) is 0:
        return [], []
    
    return contained[:, 0], contained[:, 1]

def sub_set(train, predicted):
    return list(set(predicted) - set(train))

def inter_set(train, predicted):
    return list(set(predicted) & set(train))
    
def clean_txt(string):
    return string.replace(r'(', r'\(').replace(r')', r'\)')


def get_contained_list(train: dict, predicted: dict):
    contains_df = pd.DataFrame()
    for page_id, values in train.items():
        if not predicted.get(page_id):
            continue
        true_list, predicted_list = contains_set(values, predicted[page_id])
        contains_df = contains_df.append(pd.DataFrame({"page_id": page_id, "true": true_list, "predicted": predicted_list}))
    
    return contains_df.dropna().reset_index(drop=True)

def get_match_list(train: dict, predicted: dict):
    match_df = pd.DataFrame()
    for page_id, values in train.items():
        if not predicted.get(page_id):
            continue
        match_df = match_df.append(pd.DataFrame(inter_set(values, predicted[page_id])))
    
    return match_df.dropna().reset_index(drop=True)

def get_false_list(train: dict, predicted: dict):
    false_df = pd.DataFrame()
    for page_id, values in train.items():
        if not predicted.get(page_id):
            continue
        
        false_df = false_df.append(pd.DataFrame({"page_id": page_id, "predicted": sub_set(values, predicted[page_id])}))
    
    return false_df.dropna().reset_index(drop=True)


## 用途

In [5]:
train_use_dict = shinra_util.train2dict(train_raw, '用途')
contained_use_df = get_contained_list(train_use_dict, predicted_use).reset_index(drop=True)
contained_use_df.to_csv("../analysis/predicted_use_with_contained_train.csv", index=False)

In [6]:
print("Number of training data:", len(shinra_util.flatten(train_use_dict.values())))
print("Number of predicted data in training data:", 
      len(shinra_util.flatten([predicted_use[_id] for _id in train_use_dict.keys() if predicted_use.get(_id)])))
print("完全一致しているデータの数:", len(get_match_list(train_use_dict, predicted_use)))

print("部分一致している抽出データの数:", len(contained_use_df[~contained_use_df.duplicated(["page_id", "predicted"])]))
contained_use_in_train = contained_use_df[contained_use_df.predicted.str.len() < contained_use_df.true.str.len()]
contained_use_in_predicted = contained_use_df[contained_use_df.predicted.str.len() > contained_use_df.true.str.len()]
print("Number of contained in training data:", len(contained_use_in_train[~contained_use_in_train.duplicated(["page_id", "predicted"])]))
print("Number of contained in predicted data:", len(contained_use_in_predicted[~contained_use_in_predicted.duplicated(["page_id", "true"])]))

Number of training data: 2326
Number of predicted data in training data: 1606
完全一致しているデータの数: 410
部分一致している抽出データの数: 526
Number of contained in training data: 348
Number of contained in predicted data: 258


In [22]:
train_use_dict["10166"]

['冷蔵機・冷凍機の冷媒',
 '銀めっきの還元剤',
 'バーチ還元の溶媒',
 '推進剤',
 '気付け薬',
 '動力源',
 '人工衛星などの宇宙開発用機器の冷却',
 '虫刺され用の外用薬の成分',
 '化学工業では基礎的な窒素源',
 '食品添加物',
 '基礎化学品',
 'チッソ肥料の原料',
 'パンや洋菓子などの生地の膨張剤',
 '硝酸などの基礎化学品',
 '硫安などチッソ肥料の原料',
 '食物の風味']

In [21]:
predicted_use["10166"]

['パンや洋菓子などの生地の膨張剤',
 '馬車鉄道の代わり',
 '圧縮空気や蒸気の代わり',
 'NH3ルテニウム触媒',
 '燃料電池やXLR99',
 '銀鏡反応',
 '日本薬局方一部医薬品',
 '分子式',
 '活性炭',
 '虫刺され用の外用薬の成分',
 '10.5%のアンモニア水溶液',
 'バーチ還元の溶媒',
 '冷凍機の冷媒',
 '銀めっきの還元剤',
 '作動流体',
 'ルテニウム触媒']

In [9]:
# FPかつ学習データにも含まれていない用途の一覧
false_use_df = get_false_list(train_use_dict, predicted_use).reset_index(drop=True)
pd.merge(
    false_use_df
    , contained_use_df[~contained_use_df.duplicated(["page_id", "predicted"])]
    , on=["page_id", "predicted"]
    , how='left'
).pipe(lambda x: x[x.true.isna()])[["page_id", "predicted"]]
#.to_csv("../analysis/completely_false_predicted_use.csv", index=False)

,page_id,predicted
1,1385415,脂入りはんだのフラックスの主要成分
3,1023103,フレーバー
4,1023103,ラベンダー
6,1023103,ベルガモット様の芳香
8,1024773,真空昇華
10,1024773,吸着クロマトグラフィー
11,10166,作動流体
12,10166,銀鏡反応
13,10166,日本薬局方一部医薬品
14,10166,10.5%のアンモニア水溶液


In [10]:
# 「や」が含まれているデータの数

# トレーニングデータデータには含まれているが，抽出データには含まれていない場合
ya_in_train = \
contained_use_df[~contained_use_df.duplicated(["page_id", "true"])].pipe(
    lambda x: x[x.true.str.contains(r'.+や.+') & ~x.predicted.str.contains(r'.+や.+')]
).true.unique()

# 抽出データには含まれているが，トレーニングデータには含まれていない場合
ya_in_predicted = \
contained_use_df[~contained_use_df.duplicated(["page_id", "predicted"])].pipe(
    lambda x: x[~x.true.str.contains(r'.+や.+') & x.predicted.str.contains(r'.+や.+')]
).predicted.unique()

print("Num of contained \"や\" in train:", len(ya_in_train))
print("Num of contained \"や\" in predicted:", len(ya_in_predicted))

Num of contained "や" in train: 20
Num of contained "や" in predicted: 49


In [11]:
ya_in_train

array(['ビタミンAやビタミンEのようなテルペノイドの部分骨格を持つ医薬品の原料',
       'メタノールやエチレングリコールを誤飲した場合の解毒剤', '水素添加や水素化分解の触媒',
       'HEMTやHBT等の超高速半導体素子の基板', 'ウマや他のウマ科動物で一般的に使用される非ステロイド性抗炎症薬、解熱薬、鎮痛薬',
       '高血圧や鬱血性心不全の治療',
       'エポキシ化やジヒドロキシ化による二重結合の酸化は、元のカルボニル基のアルファ位にヒドロキシ基を導入するのに用いられる。',
       'グレープフルーツや夏みかんの強烈な酸味を中和させる', 'セラミックスや触媒、高純度酸化マンガンの製造',
       '精神科治療において躁状態や双極性障害に使用される', '腸内善玉菌を増やす効果',
       '硝酸ガドリニウムは特殊なガラスやセラミックスの製造において、蛍光体となる他のガドリニウム化合物の出発物質としても利用される',
       'ペーパークトマトグラフィーにおいて、還元糖や、配糖体の糖鎖部分の検出に利用することもできる。',
       'アゾ染料や、ナフトール塗料の合成原料',
       '[Co(NH3)6]3+は構造生物学で、特に陽イオンによってリン酸骨格の三次構造（English版）が安定化されるDNAやRNAについて、X線やNMRで[5]でそれらの構造を解析するために用いられる',
       '水分を保持しやすく、冷凍しても変質しなくなるほか、成形後の製品の食感を保持する', '食品添加物や工業製品の着色',
       '工業的にはアルプラゾラムやメトヘキシタール、フェノプロフェンといった薬を合成する際に用いられる。', 'ウイルスや原生動物感染症',
       'ふけ防止用シャンプーや育毛剤に有効成分として添加'], dtype=object)

In [12]:
ya_in_predicted

array(['睡眠薬や脱法ドラッグ', '蛍光顕微鏡やフローサイトメトリー', '毒殺や自殺', '多く調合香料や食品香料',
       '外用剤や化粧品等', '金属装飾品やガラスレンズ', '除氷剤や融雪剤', 'pH調整剤や酸味料', '医薬品原料や潤滑油添加剤',
       '乳化剤や発泡剤', '滴定やpH試験紙等', '風邪やアレルギーの諸症状', '触媒や酸化剤', '抗菌剤や防腐剤',
       'シャンプーや化粧品', '農薬や動物用医薬品', '躁病や双極性障害の治療', '殺菌剤や消臭剤',
       '媒染や機能性セラミックスの製造', '有機化学の試薬やフッ素化剤のほか', '合成樹脂の製造原料のほか防錆や繊維の染色',
       '解熱薬や抗炎症薬', '天然の炭酸飲料やキャンディ', '健康食品やスナック菓子類', '脱毛剤や発光塗料',
       '洗剤や洗剤の補助', '安定性リンスや化粧品', '瓶の密封せっけんの包み紙皮膜剤チョコレートお菓子や野菜や果物の天然皮膜剤',
       '香水や防虫剤', 'ガラスの製造や陶磁器の釉薬', '総合感冒薬や鎮痛薬', '一般消費者向けの総合感冒薬や鎮痛薬',
       '皮膚や眼のアレルギー症状やアレルギー性鼻炎', '悪性腫瘍である瀰漫性大細胞型B細胞性リンパ腫や多発性骨髄腫の治療',
       '洗顔料やシャンプー', 'レーザー色素やサンスクリーン剤の原料', 'うがい薬や練り歯磨き', 'ガムやスナック菓子',
       '香水や香料', 'ジアジノンなどの殺虫剤やグリフォゼートなどの除草剤',
       '他のリン化合物の中間体やホーナー・ワーズワース・エモンズ反応の試薬', '難燃剤やポリ塩化ビニルの可塑剤',
       '抗炎症効果や血流改善効果', '予防や水虫の治療', '配管詰まりの洗浄剤や石鹸の材料',
       'ブラウン管用のガラスやクリスタルガラス', '火薬や爆薬', '化粧品やパック素材', '緩衝剤や乳化剤'],
      dtype=object)

* 単純な並列表現の「や」は単語で取らなければならないことがわかる

In [13]:
# 「、」が含まれているデータの数

# トレーニングデータデータには含まれているが，抽出データには含まれていない場合
touten_in_train = \
contained_use_df[~contained_use_df.duplicated(["page_id", "true"])].pipe(
    lambda x: x[x.true.str.contains(r'.+、.+') & ~x.predicted.str.contains(r'.+、.+')]
).true.unique()

# 抽出データには含まれているが，トレーニングデータには含まれていない場合
touten_in_predicted = \
contained_use_df[~contained_use_df.duplicated(["page_id", "predicted"])].pipe(
    lambda x: x[~x.true.str.contains(r'.+、.+') & x.predicted.str.contains(r'.+、.+')]
).predicted.unique()

print("Num of contained \"、\" in train:", len(touten_in_train))
print("Num of contained \"、\" in predicted:", len(touten_in_predicted))

Num of contained "、" in train: 58
Num of contained "、" in predicted: 0


In [14]:
touten_in_train

array(['ラッカー、ワニス、石鹸や樹脂酸金属塩の製造', '造影は加齢黄斑変性、糖尿病網膜症、眼内の炎症・腫瘍などの識別に利用される',
       '抗悪性腫瘍剤（シスプラチン等）投与および放射線照射に伴う消化器症状（悪心、嘔吐）',
       '医薬、農薬、合成樹脂、界面活性剤、ロケット燃料など化成品の原料',
       '維管束を大きくして大きなブドウを作ったり、種なしブドウを作ったり', '統合失調症、躁病の治療薬',
       '半導体レーザー、光変調器、光増幅器、光導波路、発光ダイオード、受光素子等の各種光通信用デバイスの基板',
       '亜鉛、アルミニウムなどの金属の防蝕剤',
       '実験室では、ラウリル硫酸ナトリウムはポリアクリルアミドゲル電気泳動(SDS-PAGE)用にタンパク質を処理するために使用される。',
       '抗悪性腫瘍剤（シスプラチン等）投与に伴う消化器症状（悪心、嘔吐）',
       'ウマや他のウマ科動物で一般的に使用される非ステロイド性抗炎症薬、解熱薬、鎮痛薬',
       '室内環境におけるハエ、カ、ゴキブリに対して用いられる', '顕微鏡検査の際に細胞質、膠原線維、筋線維の染色に用いられる',
       '高血圧、心不全、糖尿病で腎保護作用を示す', '厨房用、給湯用、空調用の熱源',
       '実験器具の強力な洗浄液として、不溶性の有機化合物などの分解除去に用いられる',
       'エポキシ化やジヒドロキシ化による二重結合の酸化は、元のカルボニル基のアルファ位にヒドロキシ基を導入するのに用いられる。',
       'ガラス器具をシラン化し、表面の親油性を向上させる',
       'ケイ素樹脂やピロリジノン、アクリロニトリル製造時の重合反応を始めとする重合・縮合反応',
       '青酸の四量体、ポリピロリドン、ポリペプチドなど様々な有機窒素化合物製造用の触媒',
       'セラミックスや触媒、高純度酸化マンガンの製造', '接木接着部の乾燥防止、保護',
       '心身症や神経症における不安、緊張、抑うつ、睡眠障害',
       '燃焼開始時の燃料タンクからの蒸気を抑えるため、F-16にもハロン1301が用いられている', '外

## 種類

In [15]:
train_type_dict = shinra_util.train2dict(train_raw, '種類')
contained_type_df = get_contained_list(train_type_dict, predicted_type)

In [16]:
train_type_dict["878888"]

['アンモニウム塩']

In [31]:
predicted_type["878888"]

['アミノ酸', '尿素', 'アンモニウム塩', 'タンパク質']

In [17]:
print("Number of training data:", len(shinra_util.flatten([v for v in train_type_dict.values()])))
print("Number of predicted data in training data:", 
      len(shinra_util.flatten([predicted_type[_id] for _id in train_type_dict.keys() if predicted_type.get(_id)])))
print("Number of contained in training data:", len(contained_type_df[~contained_type_df.duplicated(["page_id", "predicted"])]))
print("完全一致しているデータの数:", len(get_match_list(train_type_dict, predicted_type)))

Number of training data: 755
Number of predicted data in training data: 722
Number of contained in training data: 25
完全一致しているデータの数: 331


In [18]:
false_type_df = get_false_list(train_type_dict, predicted_type)
pd.merge(
    false_type_df
    , contained_type_df[~contained_type_df.duplicated(["page_id", "predicted"])]
    , on=["page_id", "predicted"]
    , how='left'
).pipe(lambda x: x[x.true.isna()])[["page_id", "predicted"]]
#.to_csv("../analysis/completely_false_predicted_production.csv", index=False)

,page_id,predicted
0,1385415,アルコール
1,1385415,エーテル
3,10166,塩基
4,10166,アンモニウム塩
5,10166,塩
6,1041689,色素
7,1064260,有機アルミニウム化合物
8,1095173,アニオン
9,1095173,無機化合物
10,1095173,シアン化物


## 製造方法

In [23]:
train_production_dict = shinra_util.train2dict(train_raw, '製造方法')
contained_production_df = get_contained_list(train_production_dict, predicted_production)
#.to_csv("../analysis/predicted_production_with_contained_train.csv", index=False)

In [20]:
train_production_dict["3368147"]

['パゴダン異性体を経由する',
 '合成の第一歩として[7]、シクロペンタジエン2分子1をナトリウム（シクロペンタジエニル錯体を形成する）とヨウ素存在下でカップリングさせ、ジヒドロフルバレン（English版）2を得る。',
 '次にタンデム（English版）ディールス・アルダー反応によりアセチレンジカルボン酸ジメチル3ペンタジエン・アセチレン・ペンタジエンの順に反応させ、対称な付加体4を得る。',
 'この反応時には等量のペンタジエン・ペンタジエン・アセチレンの順に反応した非対称な化合物 (4b) も生じるのでこれを除去する。',
 '次に[8]、ヨウ素を一時的にヨードラクトン化反応（English版）により導入し、二酸4をジラクトン5に転換する。',
 'エステル結合が次にメタノールにより切断され、ハロヒドリン（English版）6を生じる。',
 'アルコール部をジョーンズ酸化によりケトン化し7が得られ、ヨード基を銅亜鉛偶（English版）により還元し8を得る。',
 '最後の6つの炭素を、アリルトリメチルシラン9とn-ブチルリチウムから生じるカルバニオン10をケトン基に求核付加反応させることにより導入する。',
 '次に、ビニルシラン（English版）11を酢酸中の過酢酸とラジカル置換させてジラクトン12を得て、五酸化二リンにより分子内フリーデル・クラフツ反応でジケトン13にする。',
 'この分子は必要な20の炭素原子を全て持っており、残り5つの炭素-炭素結合の生成に有利な対称性を持っている。',
 '化合物13の二重結合をパラジウム炭素による水素化により還元し14を得、ケト基を水素化ホウ素ナトリウムによりアルコール化して15を得る。',
 'このとき生じたヒドロキシ基を、ジラクトン化16したのち、塩化トシルを用いて求核置換反応により塩素に置換して17を得る。',
 '最初のC-C結合生成反応はバーチ還元の一種（リチウム、アンモニア）で、生成物は即座にクロロメチルフェニルエーテル（English版）に捕獲される[3]。',
 '化合物17の残りの塩素原子は単純に還元される。',
 'このように一時的に置換基を追加することで後のステップでエノール化が起こることを防ぐ。',
 '新たに形成されたケト基は、光化学的ノリッシュ反応によるさらなるC-C結合生

In [21]:
print("Number of contained in training data:", len(contained_production_df[~contained_production_df.duplicated(["page_id", "predicted"])]))

Number of contained in training data: 230


In [22]:
print("Number of predicted data in training data:"
      , len(shinra_util.flatten([predicted_production[_id] for _id in train_production_dict.keys() if predicted_production.get(_id)])))

Number of predicted data in training data: 324


In [26]:
false_production_df = get_false_list(train_production_dict, predicted_production)
pd.merge(
    false_production_df
    , contained_production_df[~contained_production_df.duplicated(["page_id", "predicted"])]
    , on=["page_id", "predicted"]
    , how='left'
).pipe(lambda x: x[x.true.isna()])[["page_id", "predicted"]]
#.to_csv("../analysis/completely_false_predicted_production.csv", index=False)

,page_id,predicted
15,1270151,ジヒドロキシアセトンリン酸（ジヒドロキシアセトンリンさん、英: Dihydroxyaceto...
20,1278258,またオイゲノールやスチルベン、クマリンを合成する原料になる。
30,1383393,炭酸銅(II)の正塩 CuCO3 は知られていない、とする辞典もあるが、高温高圧のCO2雰囲...
34,1470754,触媒量の塩化アルミニウムのもとに三塩化リンと単体硫黄を反応させて、あるいは五硫化二リンと五塩...
36,1490225,1-デオキシ-D-キシルロース-5-リン酸（1-デオキシ-D-キシルロース-5-リンさん、1...
40,1565134,ゆえに濃縮することはできず、商業的に合成することはできない。
45,160786,次世代のスチレン製造法として、トルエンとメタノールに塩基性ゼオライト触媒を作用させる方法が研...
58,1653935,六水和物は赤色の固体で 74 ℃ まで加熱すると緑色の酸化物に分解するため、無水物は水和物の...
61,1658717,生成物は、フッ素と乾燥ヨウ化パラジウム(II)またはヨウ化カリウムを交互に加えることにより、...
69,1690387,実験室レベルでのジシアンの合成法の1つは、シアン化水銀(II)の熱分解である。


In [27]:
# 句点の有無でFalseとなったデータの数
contained_production_df[
    contained_production_df.true.str.replace('。$', '') == contained_production_df.predicted.str.replace('。$', '')
].shape[0]

33

## 原材料

In [28]:
train_material_dict = shinra_util.train2dict(train_raw, '原材料')

In [27]:
predicted_production["1162928"]

['ベンゼンジアゾニウムカチオンに対しアジ化物イオンを作用させると、窒素ガスを放出してアジ化ベンゼンが生成することが知られていた。',
 'アリールペンタゾールはジアゾニウム塩とアジ化物イオンの低温下での反応で合成できる。']

In [26]:
predicted_material["1162928"]

['アジ化物イオン',
 '窒素ガス',
 'ベンゼンジアゾニウムカチオン',
 'アジ化ベンゼン',
 'アジ化物イオン',
 'アリールペンタゾール',
 'ジアゾニウム塩']

In [30]:
train_material_dict["1162928"]

['ジアゾニウム塩', 'アジ化物イオン', 'アジ化ベンゼン', 'ベンゼンジアゾニウムカチオン']

## 特性

In [38]:
train_characteristic_dict = shinra_util.train2dict(train_raw, '特性')
contained_characteristic_df = get_contained_list(train_characteristic_dict, predicted_characteristic)
#.to_csv("../analysis/predicted_production_with_contained_train.csv", index=False)

In [52]:
train_characteristic_dict["205750"]

['酸化性',
 '毒性',
 '環境毒性',
 '劇物',
 '発がん性',
 '橙赤色',
 '柱状の結晶',
 '赤橙色の結晶',
 '水に可溶',
 'エタノールに不溶']

In [53]:
predicted_characteristic['205750']

['毒性', '橙赤色', '結晶', '橙色', '発がん性']

In [31]:
print("Number of training data:", len(shinra_util.flatten([v for v in train_characteristic_dict.values()])))
print("Number of predicted data in training data:", 
      len(shinra_util.flatten([predicted_characteristic[_id] for _id in train_characteristic_dict.keys() if predicted_characteristic.get(_id)])))
print("Number of contained in training data:", len(contained_characteristic_df[~contained_characteristic_df.duplicated(["page_id", "predicted"])]))
print("完全一致しているデータの数:", len(get_match_list(train_characteristic_dict, predicted_characteristic)))

Number of training data: 1351
Number of predicted data in training data: 992
Number of contained in training data: 524
完全一致しているデータの数: 195


In [32]:
false_characteristic_df = get_false_list(train_characteristic_dict, predicted_characteristic)
pd.merge(
    false_characteristic_df
    , contained_characteristic_df[~contained_characteristic_df.duplicated(["page_id", "predicted"])]
    , on=["page_id", "predicted"]
    , how='left'
).pipe(lambda x: x[x.true.isna()])[["page_id", "predicted"]]
#.to_csv("../analysis/completely_false_predicted_characteristic.csv", index=False)

,page_id,predicted
1,1052083,紫色
9,1024773,緑色
10,1027018,赤褐色
11,1027018,橙色
12,1027018,黄色
13,10166,白色
15,10166,液体
18,1050477,超短時間作用性
22,1083386,活性
23,11006,黄色
